# **Traffic Sign Recognition** 

---

**Build a Traffic Sign Recognition Project**

The goals / steps of this project are the following:
* Load the data set (see below for links to the project data set)
* Explore, summarize and visualize the data set
* Design, train and test a model architecture
* Use the model to make predictions on new images
* Analyze the softmax probabilities of the new images
* Summarize the results with a written report


[//]: # (Image References)

[image1]: ./examples/visualization.jpg "Visualization"
[image2]: ./examples/grayscale.jpg "Grayscaling"
[image3]: ./examples/random_noise.jpg "Random Noise"
[image4]: ./examples/placeholder.png "Traffic Sign 1"
[image5]: ./examples/placeholder.png "Traffic Sign 2"
[image6]: ./examples/placeholder.png "Traffic Sign 3"
[image7]: ./examples/placeholder.png "Traffic Sign 4"
[image8]: ./examples/placeholder.png "Traffic Sign 5"

## Rubric Points
### Rubric points [rubric points](https://review.udacity.com/#!/rubrics/481/view).

#### Files Submitted 

| Files         		|     Submitted	        					| 
|:---------------------:|:---------------------------------------------:| 
| Ipython notebook with code | Yes   							| 
| HTML output of the code | Yes 	|
| A writeup report (either pdf or markdown) | Yes (Markdown) |


---
### Writeup / README

### Data Set Summary & Exploration

#### 1. Provide a basic summary of the data set. In the code, the analysis should be done using python, numpy and/or pandas methods rather than hardcoding results manually.

I used the pandas library to calculate summary statistics of the traffic
signs data set:

* The size of training set is ?
* The size of the validation set is ?
* The size of test set is ?
* The shape of a traffic sign image is ?
* The number of unique classes/labels in the data set is ?

#### 2. Include an exploratory visualization of the dataset.

Here is an exploratory visualization of the data set. It is a bar chart showing how the data ...

![alt text][image1]

### Design and Test a Model Architecture

#### 1. Describe how you preprocessed the image data. What techniques were chosen and why did you choose these techniques? Consider including images showing the output of each preprocessing technique. Pre-processing refers to techniques such as converting to grayscale, normalization, etc. (OPTIONAL: As described in the "Stand Out Suggestions" part of the rubric, if you generated additional data for training, describe why you decided to generate additional data, how you generated the data, and provide example images of the additional data. Then describe the characteristics of the augmented training set like number of images in the set, number of images for each class, etc.)

As a first step, I decided to convert the images to grayscale because ...

Here is an example of a traffic sign image before and after grayscaling.

![alt text][image2]

As a last step, I normalized the image data because ...

I decided to generate additional data because ... 

To add more data to the the data set, I used the following techniques because ... 

Here is an example of an original image and an augmented image:

![alt text][image3]

The difference between the original data set and the augmented data set is the following ... 


#### 2. Describe what your final model architecture looks like including model type, layers, layer sizes, connectivity, etc.) Consider including a diagram and/or table describing the final model.

My final model consisted of the following layers:

| Layer         		|     Description	        					| 
|:---------------------:|:---------------------------------------------:| 
| Input         		| 32x32x3 RGB image   							| 
| Convolution 3x3     	| 1x1 stride, same padding, outputs 32x32x64 	|
| RELU					|												|
| Max pooling	      	| 2x2 stride,  outputs 16x16x64 				|
| Convolution 3x3	    | etc.      									|
| Fully connected		| etc.        									|
| Softmax				| etc.        									|
|						|												|
|						|												|
 


#### 3. Describe how you trained your model. The discussion can include the type of optimizer, the batch size, number of epochs and any hyperparameters such as learning rate.

To train the model, I used an ....

#### 4. Describe the approach taken for finding a solution and getting the validation set accuracy to be at least 0.93. Include in the discussion the results on the training, validation and test sets and where in the code these were calculated. Your approach may have been an iterative process, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think the architecture is suitable for the current problem.

My final model results were:
* training set accuracy of ?
* validation set accuracy of ? 
* test set accuracy of ?

If an iterative approach was chosen:
* What was the first architecture that was tried and why was it chosen?
* What were some problems with the initial architecture?
* How was the architecture adjusted and why was it adjusted? Typical adjustments could include choosing a different model architecture, adding or taking away layers (pooling, dropout, convolution, etc), using an activation function or changing the activation function. One common justification for adjusting an architecture would be due to overfitting or underfitting. A high accuracy on the training set but low accuracy on the validation set indicates over fitting; a low accuracy on both sets indicates under fitting.
* Which parameters were tuned? How were they adjusted and why?
* What are some of the important design choices and why were they chosen? For example, why might a convolution layer work well with this problem? How might a dropout layer help with creating a successful model?

If a well known architecture was chosen:
* What architecture was chosen?
* Why did you believe it would be relevant to the traffic sign application?
* How does the final model's accuracy on the training, validation and test set provide evidence that the model is working well?
 

### Test a Model on New Images

#### 1. Choose five German traffic signs found on the web and provide them in the report. For each image, discuss what quality or qualities might be difficult to classify.

Here are five German traffic signs that I found on the web:

![alt text][image4] ![alt text][image5] ![alt text][image6] 
![alt text][image7] ![alt text][image8]

The first image might be difficult to classify because ...

#### 2. Discuss the model's predictions on these new traffic signs and compare the results to predicting on the test set. At a minimum, discuss what the predictions were, the accuracy on these new predictions, and compare the accuracy to the accuracy on the test set (OPTIONAL: Discuss the results in more detail as described in the "Stand Out Suggestions" part of the rubric).

Here are the results of the prediction:

| Image			        |     Prediction	        					| 
|:---------------------:|:---------------------------------------------:| 
| Stop Sign      		| Stop sign   									| 
| U-turn     			| U-turn 										|
| Yield					| Yield											|
| 100 km/h	      		| Bumpy Road					 				|
| Slippery Road			| Slippery Road      							|


The model was able to correctly guess 4 of the 5 traffic signs, which gives an accuracy of 80%. This compares favorably to the accuracy on the test set of ...

#### 3. Describe how certain the model is when predicting on each of the five new images by looking at the softmax probabilities for each prediction. Provide the top 5 softmax probabilities for each image along with the sign type of each probability. (OPTIONAL: as described in the "Stand Out Suggestions" part of the rubric, visualizations can also be provided such as bar charts)

The code for making predictions on my final model is located in the 11th cell of the Ipython notebook.

For the first image, the model is relatively sure that this is a stop sign (probability of 0.6), and the image does contain a stop sign. The top five soft max probabilities were

| Probability         	|     Prediction	        					| 
|:---------------------:|:---------------------------------------------:| 
| .60         			| Stop sign   									| 
| .20     				| U-turn 										|
| .05					| Yield											|
| .04	      			| Bumpy Road					 				|
| .01				    | Slippery Road      							|


For the second image ... 

### (Optional) Visualizing the Neural Network (See Step 4 of the Ipython notebook for more details)
#### 1. Discuss the visual output of your trained network's feature maps. What characteristics did the neural network use to make classifications?




# **Finding Lane Lines on the Road** 

---

#### Finding Lane Lines on the Road

##### Goal:

Lane recognition plays an important role in ADAS and self driving car.
This project demonstrates 

1. a simple lane recognition algorithm given a road image.
2. applying the algorithm to frames of given videos.

---

#### Approach:

#### 1. Lane recognition pipeline

Following image shows the conceptual pipeline of the lane recognition 

![](./imgs/ProcConcept.png)
(Start from right to left)

##### Base steps

* Given a road image, we apply gaussian filter to reduce noise
* Apply generate canny edge detection 
* Set a image mask to consider only the ROI ( region of interest )
* Apply hough transformation to get line segments 
* Display overlayed hough lines as lane see draw_lines() function.


##### Drawing extrapolated lane line

Given the base steps, we get disconnected lane lines.
To get the extrapolated lane we apply following steps 
( implemented in draw_lines() see P1_solution.ipynb )

* Rough clustering of hough lines 

![](./imgs/HoughLines.png)

(individual hough lines visualized in different colors)

In this project, we apply a simple rule to cluster candidate lines of left_lane and right_rane 

As the origin of the image coordinate starts from top left corner

Candidates for the left lanes are those with slope under 0, and likewise, cadidates for the right lanes are those with slope greater than 0. 

```python
    # 1. Cluster lines into two groups ( left and right )
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = get_slope(x1,y1,x2,y2)
            if slope < 0 : 
                left_lane_lines.append([x1,y1,x2,y2,slope])
            else :
                right_lane_lines.append([x1,y1,x2,y2,slope])
```

* Filter out outliers 

Lanes can be considered to be dominant lines in an image.
Therefore, majority of the houghline segment should have similar slope.

While driving on the road the lanes can be considered to be diagonal lines. Thus, we also filter out dominant horizontal hough lines by applying filtering of ( 0 ~ 15 degree ) slopes.

```python
def get_extrapolated_line(lines,h,w):
    '''
    lines = [[ x1,y1,x2,y2,slope]...]
    '''
    l = np.array(lines) 
    # filter slopes in range of ( 0 ~ 15 degree)
    l = l[abs(l[:,4]) > 0.15]
    :
```

Then we compute mean slope and standard deviation and just filter out line segment that are outside of the std.

```python
def get_extrapolated_line(lines,h,w):
      :
    lslopes = l[:,4].ravel()
    lslope_mean = lslopes.mean()
    lslope_std = lslopes.std()

    l = l[abs(l[:,4] - lslope_mean) < lslope_std ]
```

* Applying line model ( y = ax + b )

slope : Now given filtered out candidate lane lines, we calculate average slope. 

```python
avg_slope = l[:,4].ravel().mean()
```

near center x and y point : given hough line candidates we pick the nearest point to the center of the image.

```phython
    center_y = l[:,[1,3]].ravel().min()
    center_x = l[:,[0,2]].ravel().min() if avg_slope >= 0 else l[:,[0,2]].ravel().max()
```

Given line equation y = ax + b, now we can calculate the other end point of the line lying on the bottom image. 

```python
def get_extrapolated_line(lines,h,w):
     :
    #####################################################
    # Origin (0,0) : upper left corner 
    # right_lane = slope > 0  |  left_lane = slope < 0 
    # y = slope * x + b 
    #####################################################
    b = center_y - (avg_slope * center_x)
    bottom_y = h # image height
    bottom_x = ( bottom_y - b ) / avg_slope 
    
    return int(bottom_x), int(bottom_y), int(center_x), int(center_y)
```

See get_extrapolated_line() function in the notebook ( P1-solution.ipynb )


* Drawing the lines on input frame image 

Given the line points from get_extrapolated_line(), draw the line using opencv line drawing function.

```python
def draw_extrapolated_lines() :
    :
    x1, y1, x2, y2 = get_extrapolated_line(right_lane_lines,img.shape[0],img.shape[1])
    #print (x1, y1, x2, y2)
    cv2.line(img, (x1, y1), (x2, y2), color_map[0], thickness=3)
    :
```

see draw_extrapolated_lines() function in the notebook ( P1-solution.ipynb )

Following shows the result on a sample image

![](./imgs/ExtrapolatedLanes.png)

(with extrapolated lane line)


#### 2. Optional Challenge 

As the edge detection and hough transformation works on pixel intensity, it is sensative to noise, reflection and shadow, etc.

Given a challenging sample video "test_videos_output/challenge.mp4", we demonstrate the proposed approach also works fine.

We've applied two changes 

1) We consider yellow and white component in the image to form a gray input for edge detection. 

   ( select color component in HSV color space)

```python
def process_image_ext_line_v2(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    yellow = cv2.inRange(hsv, (20, 80, 80), (25, 255, 255))
    white = cv2.inRange(hsv, (0, 0, 180), (255, 25, 255))
    gray_from_yellow_white = cv2.bitwise_or(yellow, white)

    blur_gray = gaussian_blur(gray_from_yellow_white, 7)
    edges = canny(blur_gray, 50, 150)
      :
```

2) dynamically setting the ROI polygon ( to cope with different frame size )

```python
    ratio_center_x_left = 0.47
    ratio_center_x_right = 0.51
    ratio_center_y_heighgt = 0.59
    
    center_x_left = int(imshape[1]*ratio_center_x_left)
    center_x_right= int(imshape[1]*ratio_center_x_right)
    center_y_height = int(imshape[0]*ratio_center_y_heighgt)
    
    vertices = np.array([[(0,imshape[0]),(center_x_left, center_y_height), (center_x_right, center_y_height), (imshape[1],imshape[0])]], dtype=np.int32)

```

![](./imgs/Challenge-Frame-SampleResult.png)

![](./imgs/Challenge-Frame-SampleResult2.png)


---

#### 2. Identify potential shortcomings with your current pipeline

* Setting proper RoI

This approach assumes fixed RoI ratio.
depending on driving and road condition, actual road regions can be skipped out 

* Line model 

This approach assumes lane to be a straight forward line.
Given that, it applied a very simple line model.
Naively picked lane ends ( near center of image / vanishing line ) 
Thus, it would introduce errors. 

* Noise / Shadow / Day Night condition 

As the algorithm parameters were set given test dataset,
those parameters need to be adjusted for frames from different scneario.

* And many more 


#### 3. Suggest possible improvements to your pipeline

We could apply more strong denoising filters.

Image segementation tech would help to extract real road region. 
For example we could apply color segmentation, etc. 

Given the segmented region, we could pose adjusted ROI. 

To cope with shadowing, 
we could make a region color model by cummulating region of previous frames. 

Given the hough lines, we could also cosider lane as a connected polygon of lines. 
We could apply better line fitting algorithm, etc.
